# Read In URL 
## Description  
Program to input all the URL and read in the content 


In [ ]:
import pandas as pd
path = "data/RSS_News_Feeds/"
file_name = path + "articles.xlsx"

In [ ]:
xl_file = pd.ExcelFile(file_name)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [ ]:
dfs.keys()

In [ ]:
counts = [len(dfs[k]) for k in dfs.keys()]
df_cnts = pd.DataFrame({"subject": list(dfs.keys()), "cnt": counts})

In [ ]:
filtered_subjects = df_cnts[df_cnts.cnt > 9]
filtered_subjects

## Extract URL

In [ ]:
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import time

class w_driver():
    
    def __enter__(self):
        self.driver = webdriver.Firefox()
        return self.driver

    def __exit__(self, exc_type, exc_value, traceback):
        if self.driver is not None:
            self.driver.quit()
        self.driver = None
    
def web_content(mydriver, url):
    assert (len(url) > 5), "Invalid url"
    assert (url is not None), "Invalid url"
    mydriver.get(url)
#    time.sleep(1)
    html = mydriver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',
        'div',
        'meta'
        # there may be more elements you don't want, such as "style", etc.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t.rstrip())

    return output        

In [ ]:
for subject in filtered_subjects.subject:
    print(dfs[subject].columns)

In [ ]:
inp = dfs[filtered_subjects.subject.iloc[1]]["SOURCE/LINK"][2]

l_topic = []
l_title = []
l_issue = []
l_source = []
l_pub_date = []
l_rec_date = []
l_content = []

with w_driver() as mydriver:
    for f_subject in filtered_subjects.subject:
        print(f_subject)
        for src in dfs[f_subject]['SOURCE/LINK']:
            content = ""; 
            try:
                addr = re.findall(r'(http:\/.*|https:\/.*|www\..*)$', src)[0].rstrip().lstrip()
                if addr[:4] != "http": addr = "http://" + addr
                try:
                    content = web_content(mydriver, addr)
                except:
                    pass
            except:
                pass
            l_content.append(content)
            
        l_topic = l_topic + list(dfs[f_subject]['TOPIC'])
        l_title = l_title + list(dfs[f_subject]['TITLE'])
        l_issue = l_issue + list(dfs[f_subject]['TITLE'])
        l_source = l_source + list(dfs[f_subject]['SOURCE/LINK'])
        l_pub_date = l_pub_date + list(dfs[f_subject]['DATE PUBLISHED'])
        l_rec_date = l_rec_date + list(dfs[f_subject]['DATE RECORDED'])
        
df = pd.DataFrame({"TOPIC":l_topic, "TITLE":l_title, "ISSUE":l_issue, "SOURCE":l_source, "PUB_DATE": l_pub_date, "REC_DATE": l_rec_date, "TEXT": l_content})        
df